In [1]:
import numpy 
import math
import random
import pickle
import time

### Pickling

In [2]:
def read_pickle(filename):
    infile = open(filename,'rb')
    dict_data = pickle.load(infile)
    infile.close()
    return dict_data

In [3]:
def write_pickle(dict_data, filename):
    pickling_on = open(filename+'.pickle',"wb")
    pickle.dump(dict_data, pickling_on)
    pickling_on.close()
    print('saved file')

### UUniFast and Friends
##### Code from : https://github.com/MaximeCheramy/simso/blob/master/simso/generator/task_generator.py
and https://github.com/brandenburg/schedcat/blob/master/schedcat/generator/generator_emstada.py

In [4]:
# from https://github.com/MaximeCheramy/simso/blob/master/simso/generator/task_generator.py
def UUniFastDiscard(n, u, nsets):
    '''
    n: number of tasks
    u: utilisation
    nsets: number of tasksets (instances) to be created
    
    Can be used for multiprocessor case 
    Becomes highly inefficient as u approaches n/2
    '''
    
    sets = []
    while len(sets) < nsets:
        # Classic UUniFast algorithm:
        utilizations = []
        sumU = u
        for i in range(1, n):
            nextSumU = sumU * random.random() ** (1.0 / (n - i))
            utilizations.append(sumU - nextSumU)
            sumU = nextSumU
        utilizations.append(sumU)

        # If no task utilization exceeds 1: - safety check
        if all(ut <= 1 for ut in utilizations):
            sets.append(utilizations)

    return sets


In [5]:
# https://github.com/MaximeCheramy/simso/blob/master/simso/generator/task_generator.py
def UUniFast(n, u, nsets):
    '''
    n: number of tasks
    u: utilisation
    nsets: number of tasksets (instances) to be created
    
    For uniprocessor case
    '''
    
    sets = []
    while len(sets) < nsets:
        # Classic UUniFast algorithm:
        utilizations = []
        sumU = u
        for i in range(1, n):
            nextSumU = sumU * random.random() ** (1.0 / (n - i))
            utilizations.append(sumU - nextSumU)
            sumU = nextSumU
        utilizations.append(sumU)

#         # If no task utilization exceeds 1:
#         if all(ut <= 1 for ut in utilizations):
        sets.append(utilizations)

    return sets


### There is a problem with xrange (not defined in python 3). So can it be simple changed to range?

In [6]:
# from https://github.com/brandenburg/schedcat/blob/master/schedcat/generator/generator_emstada.py

def StaffordRandFixedSum(n, u, nsets):

    #deal with n=1 case
    if n == 1:
        return numpy.tile(numpy.array([u]),[nsets,1])

    k = numpy.floor(u)
    s = u
    step = 1 if k < (k-n+1) else -1
    s1 = s - numpy.arange( k, (k-n+1)+step, step )
    step = 1 if (k+n) < (k-n+1) else -1
    s2 = numpy.arange( (k+n), (k+1)+step, step ) - s

    tiny = numpy.finfo(float).tiny
    huge = numpy.finfo(float).max

    w = numpy.zeros((n, n+1))
    w[0,1] = huge
    t = numpy.zeros((n-1,n))

    for i in numpy.arange(2, (n+1)):
        tmp1 = w[i-2, numpy.arange(1,(i+1))] * s1[numpy.arange(0,i)]/float(i)
        tmp2 = w[i-2, numpy.arange(0,i)] * s2[numpy.arange((n-i),n)]/float(i)
        w[i-1, numpy.arange(1,(i+1))] = tmp1 + tmp2;
        tmp3 = w[i-1, numpy.arange(1,(i+1))] + tiny;
        tmp4 = numpy.array( (s2[numpy.arange((n-i),n)] > s1[numpy.arange(0,i)]) )
        t[i-2, numpy.arange(0,i)] = (tmp2 / tmp3) * tmp4 + (1 - tmp1/tmp3) * (numpy.logical_not(tmp4))

    m = nsets
    x = numpy.zeros((n,m))
    rt = numpy.random.uniform(size=(n-1,m)) #rand simplex type
    rs = numpy.random.uniform(size=(n-1,m)) #rand position in simplex
    s = numpy.repeat(s, m);
    j = numpy.repeat(int(k+1), m);
    sm = numpy.repeat(0, m);
    pr = numpy.repeat(1, m);

    for i in numpy.arange(n-1,0,-1): #iterate through dimensions
        e = ( rt[(n-i)-1,...] <= t[i-1,j-1] ) #decide which direction to move in this dimension (1 or 0)
        sx = rs[(n-i)-1,...] ** (1/float(i)) #next simplex coord
        sm = sm + (1-sx) * pr * s/float(i+1)
        pr = sx * pr
        x[(n-i)-1,...] = sm + pr * e
        s = s - e
        j = j - e #change transition table column if required

    x[n-1,...] = sm + pr * s

    #iterated in fixed dimension order but needs to be randomised
    #permute x row order within each column
    
# #     Original
#     for i in xrange(0,m):
#         x[...,i] = x[numpy.random.permutation(n),i]

    for i in range(0,m):
        x[...,i] = x[numpy.random.permutation(n),i]

    return numpy.transpose(x);

In [7]:
def basic_check_Staffords_algo():
    '''
    This function does 2 simple checks on the given 'StaffordRandFixedSum' function above
    1. If sum of all utilisations of tasks in a taskset add up to total utilisation
    2. If all utilisations of all tasks in all tasksets is not more than 1
    '''
    flag = 0
    num_runs = 100
    li2 = StaffordRandFixedSum(100, 50, 100)
    
    for run in range(0,num_runs):
        
        for li in li2:
            if max(li) >= 1:
                flag = 1
                break
            if sum(li) > 51 or sum(li) < 49:
                flag = 1
                break
            if len(li) != 100:
                flag = 1
                break
        
        if flag == 1:
            return('wrong')
            break

    if flag == 0 and len(li2)==100:
        print('correct')
    else:
        print('wrong')
basic_check_Staffords_algo()

correct


### Miscellaneous Functions

In [8]:
def generate_N_E_i(tasks, edge_server):
    '''
    Needs review - num_edge_assgmts per task
    
    tasks = ['task_1', 'task_2']- in this form
    edge_server = ['edge_1', 'edge_2']   - in this form
    
    returns N_E_i and N_I_e
    '''
    N_E_i = {}
    N_I_e = {}
    
    for task in tasks:
        
#         Should it be len(edge_list) divided by 2 below?
#         num_edge_assgmts = random.randint(1, math.ceil(len(edge_server)/2))  
        num_edge_assgmts = random.randint(1, 2)  
#         num_edge_assgmts = 2   
        
#         randomly sample from edge_list
        N_E_i[task] = list(random.sample(edge_server, num_edge_assgmts))
    
    for edge in edge_server:
        N_I_e[edge] = []

        for task in N_E_i:
            if edge in N_E_i[task]:
                N_I_e[edge].append(task)
    
    return N_E_i, N_I_e

In [9]:
def set_server_variables():
    '''
    Needs review - the ranges for randomly generating values
    
    No input, please change values in the function directly
    All server related variables to be fixes across taskset instances 
    
    Note: Must only be called once throughout
    '''
    

    
    unit_b = 1
    unit_c = 1
    
    num_cloud = 5
    num_edge = 20
    
    cloud_server, edge_server = [], []
    for i in range(num_cloud):
        cloud_server.append('cloud_'+str(i+1))
    for i in range(num_edge):
        edge_server.append('edge_'+str(i+1))
    
    V_k, U_j = {}, {}
    for cloud in cloud_server:
        V_k[cloud] = random.randint(80, 100)    # need to change range
    for edge in edge_server:
        V_k[edge] = random.randint(40, 60)     # need to change range
        U_j[edge] = random.randint(40, 100)     # need to change range
    
    C_k, B_j = {}, {}
    for key in V_k.keys():
        C_k[key] = V_k[key]*unit_c
    for key in U_j.keys():
        B_j[key] = U_j[key]*unit_b
        
    alpha, beta = [], []
    for i in range(1, 1+max(U_j.values())):
        alpha.append(i)
    for i in range(1, 1+max(V_k.values())):
        beta.append(i)
    
    delta_jk = {}
    for edge in edge_server:
        for key in C_k.keys():
            if edge == key:          # need to keep tuple where both values are same edge due to Gurobi
                    delta_jk[(edge,key)] = 0
            else:
                delta_jk[(edge,key)] = random.randint(1, 10)    # change to 1-10
    
    return V_k, U_j, C_k, B_j, delta_jk, alpha, beta, cloud_server, edge_server, unit_b, unit_c

In [10]:
def generate_tau_g(tasks, delta_jk):
    '''
    tasks = ['task_1', 'task_2'] - in this form
    delta_jk = {(edge_1, cloud_1): 7, ((edge_1, cloud_2): 5)} - in this form
    
    Need to review ranges of values used to generate tau and G values
    Need to review value of c (a bigger number) like 10
    '''
    c = 15
    tau_i, G_i = {}, {}
    
    # Calculating mean delta_jk mean value and max value
    max_delta_jk = max(delta_jk.values())
    sum_delta_jk = 0
    count_delta_jk = 0
    mean_delta_jk = 0
    
    for key in delta_jk.keys():
        if key[0] != key[1]:
            sum_delta_jk += delta_jk[key]
            count_delta_jk += 1
    
    # Rounding up mean value of delta_jk by one unit
    mean_delta_jk = sum_delta_jk/count_delta_jk
    
    for task in tasks:
        G_i[task] = random.randint(10, 100) # need to review range of values
        
        rand_num = random.random()
        if rand_num > 0.5:
            tau_i[task] = random.randint(2*max_delta_jk + c, 2*max_delta_jk + c + 30) # need to review range of values
        else:
            tau_i[task] = random.randint(2*math.ceil(mean_delta_jk) + c, 2*math.ceil(mean_delta_jk) + c + 30) # need to review range of values
    
    return tau_i, G_i

### Generating Taskset Instances

#### One issue: Have to make sure num_tasks > max u_c value 

In [11]:
def generate_tasksets(early_stop):
    '''
    early stop - 1 or any other number - for testing - if 1 stop at 100 tasksets
    output - tasksets which is a dictionary
    
    tasksets:
    key - ID of of that instance
    value -  Dictionary containing tasks, G_i, tau_i, N_E_i, theta_i, S_i
    
    need to review ranges - Note: num_tasks must be greater than max u_c value 
    '''
    
    tasksets = {}
    taskset_ID = 0

    random.seed(1000)
    # Setting server related variables
    V_k, U_j, C_k, B_j, delta_jk, alpha, beta, cloud_server, edge_server, unit_b, unit_c = set_server_variables()
    
    
    # Calculating mean delta_jk mean value and max value
    max_delta_jk = max(delta_jk.values())
    sum_delta_jk = 0
    count_delta_jk = 0
    mean_delta_jk = 0
    for key in delta_jk.keys():
        if delta_jk[key] != 0:
            sum_delta_jk += delta_jk[key]
            count_delta_jk += 1

    mean_delta_jk = sum_delta_jk/count_delta_jk
    
    
    num_tasks_vary, u_c_vary, u_b_vary = [], [], []
    num_uuniform_runs = 30                              
    
    # num_tasks_vary = [40, 50, 60, 70, 80, 90, 100, 110, 120]   # need to review number of runs
    num_tasks_vary = [40, 60, 80, 100, 120]
    
    # fixing u_b_vary
    u_b_vary = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]    
    
    # Below u_c_vary is fixed
    # utilisation = 1 corresponds to min(C_k.values())
    # total utilisation is calculated accordingly
    min_C_k = min(C_k.values())
    u_c_total = 0
    for key in C_k.keys():
        u_c_total += (C_k[key] / min_C_k)
        
    for i in range (1, math.floor(u_c_total)+1, 4):     # need to review this increment used in the loop
        u_c_vary.append(i)
    if max(u_c_vary) != u_c_total:
        u_c_vary.append(u_c_total)
    
    #delete these
#     middle_UC = u_c_vary[7]
#     u_c_vary[0] = middle_UC
    
    print('Information from function generate_tasksets:')
    print('--------------------------------------------')
    print('num of uuniform runs: ', num_uuniform_runs)
    print('num_tasks_vary: ', num_tasks_vary)
    print('u_b_vary: ', u_b_vary)
    print('u_c_total: ', u_c_total)
    print('u_c_vary: ', u_c_vary)
    print()

    random.seed(500)

    for num_tasks in num_tasks_vary:
        
        tasks = []
        
        for i in range(num_tasks):
            tasks.append('task_'+str(i+1))
        
        for u_b in u_b_vary:
            for u_c in u_c_vary:
            
                
                for run in range(0, num_uuniform_runs):
                    
#                     generate mapping of tasks to edge servers for a given number of tasks
                    N_E_i, N_I_e = generate_N_E_i(tasks, edge_server)

#                     generate deadline and profit values
                    tau_i, G_i = generate_tau_g(tasks, delta_jk)
                    
                    theta_i, S_i = {}, {}

                    # initializing S_i as a dictionary of lists
                    for task in tasks:
                        S_i[task] = []
                    
                    # Finding theta_i values in this run
                    u_c_i = list(StaffordRandFixedSum(num_tasks, u_c, 1)[0])
                    for i in range(0, len(tasks)):
                        theta_i[tasks[i]] = (u_c_i[i] * min_C_k) * (tau_i[tasks[i]] - 2 * mean_delta_jk)
                    
                    # Finding S_i values in this run
                    for edge in edge_server:
                        u_b_i = list(UUniFast(len(N_I_e[edge]), u_b, 1)[0])
                        for i in range(0, len(N_I_e[edge])):
                            
                            # Note: S_i already initialised as dictionary of lists
                            S_i[N_I_e[edge][i]].append((u_b_i[i]*B_j[edge])*(tau_i[N_I_e[edge][i]] - 2*mean_delta_jk))
                    
                    # choose max S_i value for a task across all edge servers its mapped to
                    for task in tasks: 
                        max_value = max(S_i[task])
                        S_i[task] = max_value
                
                    tasksets[taskset_ID] = {'tasks':tasks,'G_i':G_i,'tau_i': tau_i,'N_E_i': N_E_i,\
                                            'theta_i':theta_i, 'S_i':S_i, 'u_b': u_b, 'u_c': u_c}
                    taskset_ID += 1

                    if((taskset_ID+1)%1000 == 0):
                        print('num of tasksets generated = ', taskset_ID+1)

                    if early_stop == 1 and taskset_ID==10:
                        print('--------------------------------------------')
                        print()
                        return tasksets, V_k, U_j, C_k, B_j, delta_jk, alpha, beta, cloud_server, edge_server, unit_b, unit_c
    print('--------------------------------------------')
    print()
    return tasksets, V_k, U_j, C_k, B_j, delta_jk, alpha, beta, cloud_server, edge_server, unit_b, unit_c

In [12]:
def create_tasksets_and_analysis(early_stop):
    '''
    early stop - 1 or any other number - for testing - if 1 stop at 10 tasksets
    output - tasksets which is a dictionary
    '''
    
    tasksets, V_k, U_j, C_k, B_j, delta_jk, alpha, beta, cloud_server, edge_server, unit_b, unit_c = generate_tasksets(early_stop)
    
    sum_delta_jk = 0
    count_delta_jk = 0
    mean_delta_jk = 0
    for key in delta_jk.keys():
        if delta_jk[key] != 0:
            sum_delta_jk += delta_jk[key]
            count_delta_jk += 1

    mean_delta_jk = sum_delta_jk/count_delta_jk
    
    min_C_k = min(C_k.values())
    results = {}
    count = 0

    for taskset_ID in tasksets.keys():
        count+=1
        results[taskset_ID] = {}
        
        tasks = tasksets[taskset_ID]['tasks']
        G_i = tasksets[taskset_ID]['G_i']
        tau_i = tasksets[taskset_ID]['tau_i']
        N_E_i = tasksets[taskset_ID]['N_E_i']
        theta_i = tasksets[taskset_ID]['theta_i']
        S_i = tasksets[taskset_ID]['S_i']
        
        num_comp_int = 0
        num_bdwth_int = 0
        for task in tasks:
            if (theta_i[task]/(tau_i[task] - 2*mean_delta_jk)) > 0.2*min_C_k:        # need to review - change 0.2
                num_comp_int += 1
            
            min_b_li = []
            for edge in N_E_i[task]:
                min_b_li.append(B_j[edge])
            min_b = min(min_b_li)
            
            if (S_i[task]/(tau_i[task] - 2*mean_delta_jk)) > 0.2* min_b:        # need to review - change 0.2
                num_bdwth_int += 1
            
        results[taskset_ID]['percent_comp_int'] = (num_comp_int/len(tasks))*100
        results[taskset_ID]['percent_bdwth_int'] = (num_bdwth_int/len(tasks))*100
        results[taskset_ID]['num_tasks'] = len(tasks)
    
    other_test_data = {}
    other_test_data['V_k'] = V_k
    other_test_data['U_j'] = U_j
    other_test_data['C_k'] = C_k
    other_test_data['B_j'] = B_j 
    other_test_data['delta_jk'] = delta_jk 
    other_test_data['alpha'] = alpha 
    other_test_data['beta'] = beta 
    other_test_data['cloud_server'] = cloud_server 
    other_test_data['edge_server'] = edge_server   
    other_test_data['unit_b'] = unit_b
    other_test_data['unit_c'] = unit_c
    return tasksets, results, other_test_data

In [13]:
def basic_analysis():
    
    print('Information from function basic_analysis()')
    print('-----------------------------------------------')
    print()
    tasksets_test, results_test, other_test_data = create_tasksets_and_analysis(1)
    print('results keys: ', results_test[0].keys())
    print('other_test_data keys: ', other_test_data.keys())
    print()
    print('number of tasksets:', len(results_test.keys()))
    print()
    
    print('percent_bdwth_int: ', end=' ')
    for key in results_test.keys():
        print(results_test[key]['percent_bdwth_int'], end=' ')
    print()
    
    print('percent_comp_int: ', end=' ')
    for key in results_test.keys():
        print(results_test[key]['percent_comp_int'], end=' ')
    print()
    
    print('num_tasks: ', end=' ')
    for key in results_test.keys():
        print(results_test[key]['num_tasks'], end=' ')
    print()
        
    print('-----------------------------------------------')
    return tasksets_test, results_test, other_test_data

### Driver Code - Create and Save

In [14]:
# tasksets, results, other_test_data = basic_analysis()

In [15]:
def main():
    tasksets, results, other_test_data = create_tasksets_and_analysis(0)
    
    print()
    print()
    print('Information from main()')
    print('-----------------------------------------------')
    print('Number of taskset: ', len(tasksets.keys()))
    print()
    
    comp_int_num = {'0-10':0, '10-20':0, '20-30':0, '30-40':0, '40-50':0, '50-60':0, '60-70':0, '70-80':0, '80-90':0,'90-100':0}
    bdwth_int_num = {'0-10':0, '10-20':0, '20-30':0, '30-40':0, '40-50':0, '50-60':0, '60-70':0, '70-80':0, '80-90':0,'90-100':0}
    
    for ID in results.keys():
        if results[ID]['percent_comp_int'] >=0 and results[ID]['percent_comp_int'] < 10:
            comp_int_num['0-10'] += 1 
            
        if results[ID]['percent_comp_int'] >=10 and results[ID]['percent_comp_int'] < 20:
            comp_int_num['10-20'] += 1 
            
        if results[ID]['percent_comp_int'] >=20 and results[ID]['percent_comp_int'] < 30:
            comp_int_num['20-30'] += 1 
            
        if results[ID]['percent_comp_int'] >=30 and results[ID]['percent_comp_int'] < 40:
            comp_int_num['30-40'] += 1 
            
        if results[ID]['percent_comp_int'] >=40 and results[ID]['percent_comp_int'] < 50:
            comp_int_num['40-50'] += 1 
            
        if results[ID]['percent_comp_int'] >=50 and results[ID]['percent_comp_int'] < 60:
            comp_int_num['50-60'] += 1 
            
        if results[ID]['percent_comp_int'] >=60 and results[ID]['percent_comp_int'] < 70:
            comp_int_num['60-70'] += 1 
            
        if results[ID]['percent_comp_int'] >=70 and results[ID]['percent_comp_int'] < 80:
            comp_int_num['70-80'] += 1 
            
        if results[ID]['percent_comp_int'] >=80 and results[ID]['percent_comp_int'] < 90:
            comp_int_num['80-90'] += 1 
            
        if results[ID]['percent_comp_int'] >=90 and results[ID]['percent_comp_int'] <= 100:
            comp_int_num['90-100'] += 1 
            

        if results[ID]['percent_bdwth_int'] >=0 and results[ID]['percent_bdwth_int'] < 10:
             bdwth_int_num['0-10'] += 1 
            
        if results[ID]['percent_bdwth_int'] >=10 and results[ID]['percent_bdwth_int'] < 20:
             bdwth_int_num['10-20'] += 1 
            
        if results[ID]['percent_bdwth_int'] >=20 and results[ID]['percent_bdwth_int'] < 30:
             bdwth_int_num['20-30'] += 1 
            
        if results[ID]['percent_bdwth_int'] >=30 and results[ID]['percent_bdwth_int'] < 40:
             bdwth_int_num['30-40'] += 1 
            
        if results[ID]['percent_bdwth_int'] >=40 and results[ID]['percent_bdwth_int'] < 50:
             bdwth_int_num['40-50'] += 1 
            
        if results[ID]['percent_bdwth_int'] >=50 and results[ID]['percent_bdwth_int'] < 60:
             bdwth_int_num['50-60'] += 1 
            
        if results[ID]['percent_bdwth_int'] >=60 and results[ID]['percent_bdwth_int'] < 70:
             bdwth_int_num['60-70'] += 1 
            
        if results[ID]['percent_bdwth_int'] >=70 and results[ID]['percent_bdwth_int'] < 80:
             bdwth_int_num['70-80'] += 1 
            
        if results[ID]['percent_bdwth_int'] >=80 and results[ID]['percent_bdwth_int'] < 90:
             bdwth_int_num['80-90'] += 1 
            
        if results[ID]['percent_bdwth_int'] >=90 and results[ID]['percent_bdwth_int'] <= 100:
             bdwth_int_num['90-100'] += 1 
            
    print('Number of Instances split by compute intensiveness:')
    for key in comp_int_num.keys():
        print(key+': ', end='')
        print(comp_int_num[key])
    print()
            
    print('Number of Instances split by bdwth intensiveness:')
    for key in bdwth_int_num.keys():
        print(key+': ', end='')
        print(bdwth_int_num[key])
    print() 

    dict_data = {'tasksets': tasksets, 'results':results, 'other_test_data':other_test_data}
    
    
    write_pickle(dict_data, 'data/experiment_data')
    return other_test_data

In [16]:
other_test_data = main()

Information from function generate_tasksets:
--------------------------------------------
num of uuniform runs:  30
num_tasks_vary:  [40, 60, 80, 100, 120]
u_b_vary:  [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
u_c_total:  35.49999999999999
u_c_vary:  [1, 5, 9, 13, 17, 21, 25, 29, 33, 35.49999999999999]

num of tasksets generated =  1000
num of tasksets generated =  2000
num of tasksets generated =  3000
num of tasksets generated =  4000
num of tasksets generated =  5000
num of tasksets generated =  6000
num of tasksets generated =  7000
num of tasksets generated =  8000
num of tasksets generated =  9000
num of tasksets generated =  10000
num of tasksets generated =  11000
num of tasksets generated =  12000
--------------------------------------------



Information from main()
-----------------------------------------------
Number of taskset:  12000

Number of Instances split by compute intensiveness:
0-10: 2340
10-20: 887
20-30: 992
30-40: 1018
40-50: 1394
50-60: 1461
60-70: 1191
70-80:

In [18]:
other_test_data["C_k"]

{'cloud_1': 93,
 'cloud_2': 83,
 'cloud_3': 92,
 'cloud_4': 91,
 'cloud_5': 82,
 'edge_1': 54,
 'edge_2': 57,
 'edge_3': 44,
 'edge_4': 47,
 'edge_5': 55,
 'edge_6': 51,
 'edge_7': 54,
 'edge_8': 41,
 'edge_9': 55,
 'edge_10': 54,
 'edge_11': 53,
 'edge_12': 46,
 'edge_13': 57,
 'edge_14': 46,
 'edge_15': 45,
 'edge_16': 43,
 'edge_17': 45,
 'edge_18': 42,
 'edge_19': 50,
 'edge_20': 40}